# Connecting to a mysql server from python

First of all we'll need to install the `mysql-connector-python` package.

We can connect to the server without especifying a database, and create users, databases, and so on...

It is not necesary to provide a port for connecting. The default is 3306, but on my machine I have the test server on the 3307 port.

In [5]:
! docker pull mysql
! docker run --name some-mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=<password> -d mysql


Using default tag: latest


error during connect: This error may indicate that the docker daemon is not running.: Post "http://%2F%2F.%2Fpipe%2Fdocker_engine/v1.24/images/create?fromImage=mysql&tag=latest": open //./pipe/docker_engine: The system cannot find the file specified.
The system cannot find the file specified.


In [4]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="<password>",
    database="<database>"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")

for db in mycursor:
    print(db)

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

In [ ]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS db")

Usually when connecting to the server we'll provide the database name so all operations will be done on it.

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    port=3307,
    user="user",
    password="password",
    database="db"
)
mycursor = mydb.cursor(buffered=True)

# Table creation
In SQL databases, before starting to store data, tables must be created providing the definition of the data to manage

In [ ]:
mycursor.execute("CREATE TABLE IF NOT EXISTS customers (name VARCHAR(255), address VARCHAR(255), age INT)")

In [ ]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

Normally every table has a primary key. It should be added when creating the table, but we can also modify the table after creation

In [ ]:
mycursor.execute("ALTER TABLE customers ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")

# Inserting Data

In [ ]:
sql = "INSERT INTO customers (name, address, age) VALUES (%s, %s, %s)"

In [ ]:
val = ("Ruben", "IFISC Basement", 42)
mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record(s) inserted.")

**Important!**: Notice the statement: mydb.commit(). It is required to make the changes, otherwise no changes are made to the table.

In [ ]:
val = ("Antonia", "IFISC Basement", 30)
mycursor.execute(sql, val)

val = ("Emilio", "IFISC 2nd Floor", 89)
mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record(s) inserted.")

**Note**: rowcount only takes into account the rows affected during last execution...

A better way to insert many values: Use the **executemany** method

In [ ]:
val = [
  ('Peter', 'Lowstreet 4', 120),
  ('Amy', 'Apple st 652', 200),
  ('Hannah', 'Mountain 21', 20),
  ('Michael', 'Valley 345', 40),
  ('Sandy', 'Ocean blvd 2', 30),
  ('Betty', 'Green Grass 1', 50),
  ('Richard', 'Sky st 331', 82),
  ('Susan', 'One way 98', 240),
  ('Vicky', 'Yellow Garden 2', 10),
  ('Ben', 'Park Lane 38', 18),
  ('William', 'Central st 954', 2),
  ('Chuck', 'Main Road 989', 15),
  ('Viola', 'Sideway 1633', 20)
]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "record(s) inserted.")

# Queries

In [ ]:
mycursor.execute("SELECT name, address FROM customers") # *, LIMIT 5, ORDER BY name, DESC

mycursor.fetchall()


In [ ]:
mycursor.execute("SELECT * FROM customers WHERE address ='IFISC Basement'") # LIKE '%IFISC%', AND, OR, age>=18
mycursor.fetchall()

# Modifying records

Use the UPDATE and DELETE queries.

It is very important to remember:
 * Commit the changes
 * Provide a WHERE clause, unless you want to modify or delete all the table.


In [ ]:
mycursor.execute("DELETE FROM customers WHERE name = 'William'")

mydb.commit()

print(mycursor.rowcount, "record(s) deleted.")

In [ ]:
mycursor.execute("UPDATE customers SET address = 'IFISC Basement S-15' WHERE address = 'IFISC Basement'")

mydb.commit()

print(mycursor.rowcount, "record(s) updated.")

# Queries from more than one table: JOIN
You can combine rows from two or more tables, based on a related column between them, by using a JOIN statement

First let's create a favorite_car table and add a foreing key for that table to the customers table

In [ ]:
mycursor.execute("CREATE TABLE IF NOT EXISTS cars (brand VARCHAR(255), id INT AUTO_INCREMENT PRIMARY KEY)")
mydb.commit()

In [ ]:
mycursor.execute("ALTER TABLE customers ADD favorite_car INT")
mycursor.execute("ALTER TABLE customers ADD FOREIGN KEY (favorite_car) REFERENCES cars(id)")

Now let's populate the new table and add the information about favorite cars to our customers table

In [ ]:
val = [
  ('Seat',),
  ('BMW',),
  ('Mercedes',),
  ('Opel',),
  ('Toyota',),
  ('Suzuki',)
]
mycursor.executemany("INSERT INTO cars (brand) VALUES (%s)", val)

mydb.commit()

print(mycursor.rowcount, "record(s) inserted.")

In [ ]:
mycursor.execute("UPDATE customers SET favorite_car = 1 WHERE name = 'Emilio'")
mycursor.execute("UPDATE customers SET favorite_car = 2 WHERE name = 'Antonia' or name = 'Amy'")
mycursor.execute("UPDATE customers SET favorite_car = 3 WHERE name = 'Ruben'")
mycursor.execute("UPDATE customers SET favorite_car = 4 WHERE name = 'Susan' or name = 'Susan'")
mycursor.execute("UPDATE customers SET favorite_car = 5 WHERE name = 'Michael'")

Check that both tables have all needed data

In [ ]:
mycursor.execute("SELECT name,favorite_car FROM customers")
mycursor.fetchall()

In [ ]:
mycursor.execute("SELECT * FROM cars")
mycursor.fetchall()

Get the favorite car of every customer with a JOIN

Try also: LEFT JOIN & RIGHT JOIN

In [ ]:
sql="SELECT customers.name, cars.brand FROM customers JOIN cars ON customers.favorite_car = cars.id"# LEFT, RIGHT
mycursor.execute(sql)
mycursor.fetchall()

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS customers")
mycursor.execute("DROP TABLE IF EXISTS cars")